In [29]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import cohen_kappa_score, make_scorer

from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV, cross_val_predict


In [30]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.metrics import cohen_kappa_score, make_scorer, f1_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.base import clone

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

In [31]:
df = pd.read_csv('cosine_data.csv')
df.head()

,mode_substrt,mode_pool,median_esveg92,median_temp,median_do,median_current,median_depth,median_cond,snag_present,riprap_present,inout_present,flooded_present,cluster
0,2.0,04,3.0,24.8,6.0,0.02,1.60,287.0,1.0,0.0,0.0,1.0,5
1,2.0,04,2.0,28.1,12.8,0.06,0.45,507.0,0.0,0.0,0.0,0.0,4
2,1.0,04,2.0,25.3,7.2,0.04,0.50,561.0,0.0,0.0,0.0,0.0,4
3,1.0,04,1.0,26.7,5.8,0.02,0.60,327.0,1.0,0.0,1.0,0.0,4
4,2.0,04,1.0,21.7,7.3,0.03,0.90,470.0,0.0,0.0,1.0,0.0,4


In [32]:
X = df.drop(columns = ['cluster'])
y = df['cluster'] 

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [34]:
cat_cols = ['mode_pool', 'snag_present', 'riprap_present', 'inout_present', 'flooded_present', 'snag_present', 'mode_substrt']
num_cols = ['median_esveg92', 'median_temp', 'median_do', 'median_current', 'median_depth', 'median_cond']

scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

In [35]:
ohe = OneHotEncoder(drop = 'first', sparse_output = False).set_output(transform = 'pandas')
X_train = X_train.join(ohe.fit_transform(X_train[['mode_substrt', 'mode_pool']]))
X_test = X_test.join(ohe.transform(X_test[['mode_substrt', 'mode_pool']]))

X_train.drop(columns = ['mode_substrt', 'mode_pool'], inplace=True)
X_test.drop(columns = ['mode_substrt', 'mode_pool'], inplace=True)

In [36]:
param_grid = {
    'max_depth': [None, 2, 5, 7, 10],
    'min_samples_split': [2, 3, 5, 10, 20],
    'min_impurity_decrease': [0.0, 0.01, 0.1],
    'ccp_alpha': [0.0, 0.1, 0.2, 0.5, 0.7, 1.0] 
}

In [37]:
clf_1 = GridSearchCV(
        estimator=DecisionTreeClassifier(class_weight="balanced"),
        param_grid=param_grid,
        scoring=make_scorer(f1_score),
        cv=5)
clf_1.fit(X_train, y_train == 1)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(class_weight='balanced'),
             param_grid={'ccp_alpha': [0.0, 0.1, 0.2, 0.5, 0.7, 1.0],
                         'max_depth': [None, 2, 5, 7, 10],
                         'min_impurity_decrease': [0.0, 0.01, 0.1],
                         'min_samples_split': [2, 3, 5, 10, 20]},
             scoring=make_scorer(f1_score, response_method='predict'))

In [44]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, cohen_kappa_score

# Define your custom kappa scorer
kappa_scorer = make_scorer(cohen_kappa_score)

# Define your parameter distribution
param_dist = {
    'max_depth': [None, 2, 5, 7, 10, 15, 20],
    'min_samples_split': [2, 3, 5, 10, 20, 50, 100],
    'min_impurity_decrease': [0.0, 0.01, 0.05, 0.1, 0.2],
    'ccp_alpha': [0.0, 0.01, 0.05, 0.1, 0.5, 1.0]
}

stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Set up the RandomizedSearchCV with more iterations
random_search = RandomizedSearchCV(
    estimator=DecisionTreeClassifier(class_weight="balanced"),
    param_distributions=param_dist,
    n_iter=500,  # The number of iterations to sample
    scoring=kappa_scorer,
    cv=stratified_kfold  ,
    random_state=42  # Set a seed for reproducibility
)

# Fit the model
random_search.fit(X_train[y_train > 1], y_train[y_train > 1])

# Get the best estimator
best_model = random_search.best_estimator_

# Calculate Cohen's kappa score
kappa = cohen_kappa_score(y_train[y_train > 1], y_pred)

# Print the best kappa score
print(f"Best Kappa Score: {kappa}")

# Get the best parameters
print("Best parameters found:", random_search.best_params_)

# Make predictions on the training set
y_pred = best_model.predict(X_train[y_train > 1])

# Calculate confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train[y_train > 1], y_pred)

print("Confusion Matrix:")
print(cm)

Best Kappa Score: 0.5142033733522331
Best parameters found: {'min_samples_split': 2, 'min_impurity_decrease': 0.0, 'max_depth': 5, 'ccp_alpha': 0.0}
Confusion Matrix:
[[284  37  11  37]
 [ 59 117   5   3]
 [  4   2 110  44]
 [ 11   2   8  89]]


In [54]:
# Get predictions from clf_1 and clf_2
clf_1_test = clf_1.predict(X_test)
clf_2_test = best_model.predict(X_test)

# Assuming we want to use clf_1 where it's correct, and clf_2 where clf_1 is incorrect
combined_predictions = np.where(clf_1_test == 1, clf_1_test, clf_2_test)

In [56]:
cohen_kappa_score(y_test, combined_predictions)

0.6217438925663127

In [57]:
confusion_matrix(y_test, combined_predictions)

array([[122,   0,   0,   6,   4],
       [  1,  66,  13,   5,   7],
       [  1,  19,  23,   3,   0],
       [  7,   4,   3,  14,  12],
       [  1,   4,   3,   1,  19]], dtype=int64)